# 🐍 Workshop: Build a Coding LLM from Scratch
## Part III: Pre-Training a Mini Code Assistant
### 🎯 Focus: Pre-Training on Python Code

**Auteur :** Équipe IRA

**Date :** 1 Decemebre 2025

**Contexte :** Ce notebook démontre le **Pre-Training** d'un petit modèle de langage spécialisé pour le code Python. Nous utilisons des datasets de code réels (The Stack) et montrons tout le pipeline : dataset → tokenisation → entraînement CLM → génération.

---

## 📋 Table des matières

1. **Introduction théorique** : Qu'est-ce que le Pre-Training ?
2. **Configuration & Imports**
3. **Chargement des Datasets** (HuggingFace)
4. **Construction du Corpus Python**
5. **Tokenisation**
6. **Architecture du Mini-GPT**
7. **Boucle d'Entraînement**
8. **Visualisation de la Loss**
9. **Génération de Code**
10. **Conclusion**

---

## 🔹 Partie 1 : Introduction Théorique

### Qu'est-ce que le Pre-Training ?

Le **Pre-Training** est la phase où un modèle de langage apprend à partir de données brutes non supervisées. Pour un assistant de coding :

- **Objectif** : Apprendre la syntaxe Python, les patterns de code, les conventions
- **Méthode** : **Causal Language Modeling (CLM)** = prédire le prochain token
- **Formule** : Maximiser $P(x) = \prod_{t=1}^T P(x_t | x_{<t})$

### Pre-Training vs Fine-Tuning

| Phase | Données | Objectif | Exemple |
|-------|---------|----------|---------|
| **Pre-Training** | Code brut (GitHub, The Stack) | Apprendre la syntaxe du langage | Compléter `def fib(n):` |
| **SFT** | Paires instruction/code | Suivre des consignes | "Écris une fonction fibonacci" |
| **RLHF** | Préférences humaines | Code plus lisible, sûr | Code production-ready |

### Architecture utilisée

- **Decoder-only Transformer** (GPT-style)
- **Causal Mask** : le modèle ne voit que les tokens passés
- **Vocabulaire** : BPE tokenizer adapté au code

---

In [ ]:
# %% Cell 1: Imports et Configuration

# ============================================================================
# IMPORTS DES BIBLIOTHÈQUES NÉCESSAIRES
# ============================================================================

import torch                              # Framework principal pour le deep learning
import torch.nn as nn                     # Modules de réseaux de neurones (couches, fonctions d'activation)
from torch.nn import functional as F      # Fonctions utilitaires (softmax, cross-entropy, etc.)
from torch.utils.data import Dataset, DataLoader  # Classes pour gérer les données d'entraînement
import matplotlib.pyplot as plt           # Bibliothèque pour créer des graphiques
from tqdm.auto import tqdm               # Barres de progression pour suivre l'entraînement
import numpy as np                        # Calculs numériques et manipulation de tableaux
import os                                 # Opérations sur le système de fichiers

# ============================================================================
# CONFIGURATION POUR LA REPRODUCTIBILITÉ
# ============================================================================
# Fixer les seeds permet d'obtenir les mêmes résultats à chaque exécution
torch.manual_seed(42)     # Seed pour PyTorch (génération de nombres aléatoires)
np.random.seed(42)        # Seed pour NumPy

# ============================================================================
# DÉTECTION DU DEVICE (GPU ou CPU)
# ============================================================================
# Utiliser GPU si disponible pour accélérer l'entraînement (10-100x plus rapide)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Device utilisé : {device}")
print(f"🔥 PyTorch version : {torch.__version__}")

# ============================================================================
# HYPERPARAMÈTRES GLOBAUX DU WORKSHOP
# ============================================================================

# --- Hyperparamètres des données ---
SAMPLE_SIZE = 100000  # Nombre total de documents à extraire (80k code + 20k texte)
                       # Réduire à 10000-50000 pour un test rapide

# --- Hyperparamètres du modèle ---
BLOCK_SIZE = 256      # Longueur maximale des séquences en tokens
                       # Le modèle traite des blocs de 256 tokens à la fois

# --- Hyperparamètres d'entraînement ---
BATCH_SIZE = 16       # Nombre de séquences traitées simultanément
                       # Plus grand = plus rapide mais consomme plus de mémoire

N_EPOCHS = 3          # Nombre de passages complets sur tout le dataset
                       # 3 époques est suffisant pour ce workshop

LEARNING_RATE = 3e-4  # Taux d'apprentissage (learning rate)
                       # 0.0003 = standard pour les transformers (comme GPT)

🚀 Device utilisé : cuda
🔥 PyTorch version : 2.9.0+cu126


## 🔹 Partie 2 : Chargement des Datasets

Nous utilisons **bigcode/the-stack-smol** - un dataset de code source de haute qualité.

### 📚 À propos de The Stack

**The Stack** est une collection de 3TB de code source sous licence permissive, collecté depuis GitHub.
- **the-stack-smol** : Version réduite (~200GB) adaptée pour l'entraînement de petits modèles
- **Langages** : Python, JavaScript, Java, C++, etc.
- **Licence** : Permissive (MIT, Apache, etc.)

### 🔑 Authentification HuggingFace

Ce dataset est **gated** (accès contrôlé). Vous devez :

1. **Créer un compte** : https://huggingface.co/join
2. **Demander l'accès** : https://huggingface.co/datasets/bigcode/the-stack-smol
   - Cliquer sur "Agree and access repository"
   - Lire et accepter les conditions d'utilisation
3. **Créer un token** : https://huggingface.co/settings/tokens
   - Type : "Read"
   - Copier le token (format : `hf_xxxxxxxxxxxxx`)
4. **S'authentifier** (voir cellule suivante)

### ⚠️ Note
Pour test rapide : réduire `SAMPLE_SIZE` à 10000-50000 lignes.

In [ ]:
# %% Cell 2: Authentification et Chargement des Datasets

# ============================================================================
# IMPORTS POUR ACCÉDER AUX DATASETS HUGGINGFACE
# ============================================================================
from datasets import load_dataset    # Fonction pour charger des datasets depuis HuggingFace Hub
from huggingface_hub import login    # Fonction pour s'authentifier avec un token

print("🔑 Authentification HuggingFace...")

# ============================================================================
# AUTHENTIFICATION HUGGINGFACE
# ============================================================================
# Les datasets utilisés sont "gated" (accès contrôlé)
# Vous devez créer un token sur https://huggingface.co/settings/tokens

# Charger les variables d'environnement depuis le fichier .env
from dotenv import load_dotenv
import os

load_dotenv()  # Charge les variables depuis .env

HF_TOKEN = os.getenv('HF_TOKEN')  # Récupérer le token
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("✅ Authentification réussie !")
else:
    print("⚠️  Variable HF_TOKEN non définie dans .env")
    print("   1. Créer un token sur: https://huggingface.co/settings/tokens")
    print("   2. Ajouter dans .env: HF_TOKEN=votre_token_ici")

# ============================================================================
# DATASET 1: CODE PYTHON (THE STACK)
# ============================================================================
print("\n📥 Chargement de bigcode/the-stack-smol (Python)...")
print("⚠️  Première exécution : téléchargement + mise en cache (quelques minutes)")

# Chargement du dataset de code Python
dataset_code = load_dataset(
    "bigcode/the-stack-smol",  # Nom du dataset (3TB de code open-source)
    data_dir="data/python",     # Filtrer uniquement les fichiers Python
    split="train",              # Utiliser la partie "train" du dataset
    streaming=True              # Mode streaming = ne charge pas tout en mémoire
                                 # Permet de traiter des datasets énormes sans RAM overflow
)

print("✅ Dataset code chargé en mode streaming")

# Afficher un exemple pour vérifier
print("\n--- Aperçu du premier exemple (Code) ---")
first_code = next(iter(dataset_code))  # Prendre le premier élément
print(f"Clés disponibles : {first_code.keys()}")  # Voir quels champs sont disponibles
print(f"\nContenu (100 premiers caractères) :\n{first_code['content'][:100]}...")  # Afficher un extrait

# ============================================================================
# DATASET 2: TEXTE GÉNÉRAL (COSMOPEDIA)
# ============================================================================
print("\n\n📥 Chargement de HuggingFaceTB/smollm-corpus (cosmopedia-v2)...")

# Chargement du dataset de texte général
dataset_text = load_dataset(
    "HuggingFaceTB/smollm-corpus",  # Dataset de texte synthétique de haute qualité
    "cosmopedia-v2",                 # Subset spécifique (articles type encyclopédie)
    split="train",                   # Partie entraînement
    streaming=True                   # Mode streaming pour économiser la RAM
)

print("✅ Dataset texte chargé en mode streaming")

# Afficher un exemple
print("\n--- Aperçu du premier exemple (Texte) ---")
first_text = next(iter(dataset_text))  # Prendre le premier document
print(f"Clés disponibles : {first_text.keys()}")  # Voir les champs disponibles
print(f"\nContenu (100 premiers caractères) :\n{first_text['text'][:100]}...")  # Afficher un extrait

🔑 Authentification HuggingFace...
✅ Authentification réussie !

📥 Chargement de bigcode/the-stack-smol (Python)...
⚠️  Première exécution : téléchargement + mise en cache (quelques minutes)
✅ Dataset code chargé en mode streaming

--- Aperçu du premier exemple (Code) ---
✅ Dataset code chargé en mode streaming

--- Aperçu du premier exemple (Code) ---
Clés disponibles : dict_keys(['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'])

Contenu (100 premiers caractères) :
# Copyright 2020 gRPC authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
#...


📥 Chargement de HuggingFaceTB/smollm-corpus (cosmopedia-v2)...
Clés disponibles : dict_keys(['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'])

Contenu (100 premiers caractères) :
# Copyright 2020 gRPC authors.
#
# Licensed under the Apache License, Version 2.0 (the "Lic

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

✅ Dataset texte chargé en mode streaming

--- Aperçu du premier exemple (Texte) ---
Clés disponibles : dict_keys(['prompt', 'text', 'token_length', 'audience', 'format', 'seed_data'])

Contenu (100 premiers caractères) :
 In today's ever-evolving world, technology has become an integral part of our lives, shaping the wa...
Clés disponibles : dict_keys(['prompt', 'text', 'token_length', 'audience', 'format', 'seed_data'])

Contenu (100 premiers caractères) :
 In today's ever-evolving world, technology has become an integral part of our lives, shaping the wa...


In [ ]:
# %% Cell 3: Extraction et Construction du Corpus Mixte (Code + Texte)

# ============================================================================
# OBJECTIF: Créer un corpus mixte avec 80% code Python + 20% texte général
# ============================================================================

print(f"🔨 Construction du corpus mixte...")
print(f"   - Code Python : {int(SAMPLE_SIZE * 0.8):,} fichiers (80%)")
print(f"   - Texte général : {int(SAMPLE_SIZE * 0.2):,} documents (20%)\n")

# Initialiser deux listes pour stocker les documents
code_lines = []  # Contiendra les fichiers de code Python
text_lines = []  # Contiendra les documents de texte général

# ============================================================================
# ÉTAPE 1: EXTRACTION DU CODE PYTHON
# ============================================================================
code_target = int(SAMPLE_SIZE * 0.8)  # Calculer 80% du total (80,000 fichiers)
print(f"📝 Extraction du code Python ({code_target} fichiers)...")

# Parcourir le dataset de code en streaming
for idx, example in enumerate(tqdm(dataset_code, desc="Code Python", total=code_target)):
    content = example['content']  # Récupérer le contenu du fichier Python
    
    # Filtrer les fichiers trop courts (moins de 50 caractères)
    # Cela élimine les fichiers quasi-vides ou malformés
    if len(content) < 50:
        continue
    
    code_lines.append(content)  # Ajouter à la liste
    
    # Arrêter quand on a atteint l'objectif
    if len(code_lines) >= code_target:
        break

print(f"✅ {len(code_lines)} fichiers Python extraits")

# ============================================================================
# ÉTAPE 2: EXTRACTION DU TEXTE GÉNÉRAL
# ============================================================================
text_target = int(SAMPLE_SIZE * 0.2)  # Calculer 20% du total (20,000 documents)
print(f"\n📚 Extraction du texte général ({text_target} documents)...")

# Parcourir le dataset de texte en streaming
for idx, example in enumerate(tqdm(dataset_text, desc="Texte général", total=text_target)):
    content = example['text']  # Récupérer le contenu du document
    
    # Filtrer les documents trop courts (moins de 100 caractères)
    # Le texte a besoin d'être plus long que le code pour être significatif
    if len(content) < 100:
        continue
    
    text_lines.append(content)  # Ajouter à la liste
    
    # Arrêter quand on a atteint l'objectif
    if len(text_lines) >= text_target:
        break

print(f"✅ {len(text_lines)} documents de texte extraits")

# ============================================================================
# ÉTAPE 3: MÉLANGE ALÉATOIRE ET CONCATÉNATION
# ============================================================================
print("\n🔀 Mélange des données...")

# Importer random pour mélanger
import random
random.seed(42)  # Fixer le seed pour reproductibilité

# Combiner les deux listes
all_content = code_lines + text_lines  # [code1, code2, ..., text1, text2, ...]

# Mélanger aléatoirement pour que code et texte soient entrelacés
# Cela permet au modèle de voir des patterns variés pendant l'entraînement
random.shuffle(all_content)

# Concaténer tous les documents avec double saut de ligne comme séparateur
# "\n\n" permet de distinguer visuellement les documents dans le corpus
corpus = "\n\n".join(all_content)
total_chars = len(corpus)  # Calculer le nombre total de caractères

# ============================================================================
# ÉTAPE 4: AFFICHAGE DES STATISTIQUES
# ============================================================================
print(f"📊 Statistiques du corpus final :")
print(f"   - Total documents : {len(all_content):,}")  # Nombre total de documents
print(f"   - Code Python : {len(code_lines):,} ({len(code_lines)/len(all_content)*100:.1f}%)")
print(f"   - Texte général : {len(text_lines):,} ({len(text_lines)/len(all_content)*100:.1f}%)")
print(f"   - Total caractères : {total_chars:,}")  # Taille du corpus en caractères

# ============================================================================
# ÉTAPE 5: SAUVEGARDE DU CORPUS SUR DISQUE
# ============================================================================
corpus_file = "mini_corpus_mixed.txt"  # Nom du fichier de sortie

# Écrire le corpus dans un fichier texte
# encoding='utf-8' assure la compatibilité avec tous les caractères
with open(corpus_file, 'w', encoding='utf-8') as f:
    f.write(corpus)

# Afficher la taille du fichier créé
print(f"\n💾 Corpus sauvegardé dans {corpus_file} ({os.path.getsize(corpus_file)/1e6:.2f} MB)")

# Afficher un aperçu pour vérifier le contenu
print(f"\n--- Aperçu des 500 premiers caractères ---")
print(corpus[:500])
print("---" * 20)

🔨 Construction du corpus mixte...
   - Code Python : 80,000 fichiers (80%)
   - Texte général : 20,000 documents (20%)

📝 Extraction du code Python (80000 fichiers)...


Code Python:   0%|          | 0/80000 [00:00<?, ?it/s]

✅ 9936 fichiers Python extraits

📚 Extraction du texte général (20000 documents)...


Texte général:   0%|          | 0/20000 [00:00<?, ?it/s]

✅ 20000 documents de texte extraits

🔀 Mélange des données...
📊 Statistiques du corpus final :
   - Total documents : 29,936
   - Code Python : 9,936 (33.2%)
   - Texte général : 20,000 (66.8%)
   - Total caractères : 155,529,500
📊 Statistiques du corpus final :
   - Total documents : 29,936
   - Code Python : 9,936 (33.2%)
   - Texte général : 20,000 (66.8%)
   - Total caractères : 155,529,500

💾 Corpus sauvegardé dans mini_corpus_mixed.txt (155.81 MB)

--- Aperçu des 500 premiers caractères ---
 In the realm of speculative fiction, slipstream and new wave literature challenge traditional genre boundaries by blending elements of science fiction, fantasy, and literary fiction. These modes often engage with complex scientific and societal issues, serving as fertile ground for exploring the intersection of technology and humanity. The following course unit will delve into the concept of "urban" as it appears in the provided extract, drawing upon theories of slipstream and new wave analys

## 🔹 Partie 3 : Tokenisation

Pour le code, nous utilisons un **tokenizer BPE** (Byte-Pair Encoding).
Options :
1. **Utiliser GPT-2 tokenizer** (pré-entraîné, simple)
2. **Entraîner un tokenizer custom** avec `tokenizers`

Ici, nous utilisons **GPT-2** pour simplifier.

In [ ]:
# %% Cell 4: Tokenisation avec GPT-2 Tokenizer

# ============================================================================
# OBJECTIF: Convertir le texte brut en séquence de tokens (nombres)
# ============================================================================
# La tokenisation découpe le texte en morceaux (tokens) et les convertit en IDs
# Exemple: "def add(x):" → [4299, 751, 7, 87, 2599, 60]

from transformers import GPT2Tokenizer  # Importer le tokenizer de HuggingFace

print("🔤 Chargement du tokenizer GPT-2...")

# Charger le tokenizer pré-entraîné GPT-2
# GPT-2 utilise BPE (Byte-Pair Encoding) qui fonctionne bien pour le code
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# GPT-2 n'a pas de pad token par défaut, on utilise eos_token comme pad
# pad_token est utilisé pour remplir les séquences de longueurs différentes
tokenizer.pad_token = tokenizer.eos_token

# ============================================================================
# ENCODAGE DU CORPUS EN TOKENS
# ============================================================================
print("🔄 Tokenisation du corpus...")

# Encoder tout le corpus en une liste d'IDs de tokens
# add_special_tokens=False car on ne veut pas ajouter [CLS], [SEP], etc.
encoded = tokenizer.encode(corpus, add_special_tokens=False)

# Récupérer la taille du vocabulaire (nombre total de tokens possibles)
vocab_size = tokenizer.vocab_size

print(f"✅ Tokenisation terminée")
print(f"📊 Nombre de tokens : {len(encoded):,}")  # Combien de tokens dans le corpus
print(f"📚 Taille du vocabulaire : {vocab_size:,}")  # Nombre de tokens uniques possibles (50,257)

# ============================================================================
# TEST DU TOKENIZER
# ============================================================================
# Vérifier que le tokenizer fonctionne correctement sur un exemple
print(f"\n--- Test du tokenizer ---")
test_code = "def fibonacci(n):\n    return n"

# Encoder le texte en tokens
test_encoded = tokenizer.encode(test_code)

# Décoder les tokens pour retrouver le texte original
test_decoded = tokenizer.decode(test_encoded)

# Afficher les résultats
print(f"Original : {test_code}")
print(f"Encodé   : {test_encoded}")  # Liste des IDs de tokens
print(f"Décodé   : {test_decoded}")  # Doit être identique à l'original
test_decoded = tokenizer.decode(test_encoded)
print(f"Original : {test_code}")
print(f"Encodé   : {test_encoded}")
print(f"Décodé   : {test_decoded}")

🔤 Chargement du tokenizer GPT-2...
🔄 Tokenisation du corpus...
🔄 Tokenisation du corpus...


Token indices sequence length is longer than the specified maximum sequence length for this model (51993144 > 1024). Running this sequence through the model will result in indexing errors


✅ Tokenisation terminée
📊 Nombre de tokens : 51,993,144
📚 Taille du vocabulaire : 50,257

--- Test du tokenizer ---
Original : def fibonacci(n):
    return n
Encodé   : [4299, 12900, 261, 44456, 7, 77, 2599, 198, 220, 220, 220, 1441, 299]
Décodé   : def fibonacci(n):
    return n


In [ ]:
# %% Cell 5: Création du Dataset PyTorch pour CLM

# ============================================================================
# OBJECTIF: Créer un Dataset PyTorch pour l'entraînement
# ============================================================================
# PyTorch nécessite un Dataset qui fournit des paires (input, target) pour l'entraînement
# Pour un modèle de langage: input = tokens[:-1], target = tokens[1:]
# Exemple: input=[10,20,30], target=[20,30,40] → le modèle prédit le token suivant

class CodeDataset(Dataset):
    """
    Dataset pour Causal Language Modeling (CLM)
    Chaque exemple contient une séquence de tokens et sa cible (décalée de 1)
    """
    def __init__(self, token_ids, block_size):
        """
        Args:
            token_ids (list): Liste des IDs de tokens du corpus
            block_size (int): Longueur de chaque séquence (256)
        """
        self.token_ids = token_ids
        self.block_size = block_size
        
    def __len__(self):
        """
        Nombre d'exemples disponibles
        On soustrait block_size pour avoir assez de tokens pour input ET target
        """
        return len(self.token_ids) - self.block_size
    
    def __getitem__(self, idx):
        """
        Retourne un exemple (input, target)
        Input : tokens de idx à idx+block_size
        Target : tokens de idx+1 à idx+block_size+1 (décalé de 1 position)
        """
        # Extraire la séquence d'input
        x = torch.tensor(self.token_ids[idx:idx+self.block_size], dtype=torch.long)
        
        # Extraire la séquence de target (décalée de 1)
        y = torch.tensor(self.token_ids[idx+1:idx+self.block_size+1], dtype=torch.long)
        
        return x, y

# ============================================================================
# DIVISION EN TRAIN / VALIDATION
# ============================================================================
# Convertir les tokens en tensor PyTorch
tokens = torch.tensor(encoded, dtype=torch.long)

# Diviser le corpus: 90% train, 10% validation
split_idx = int(0.9 * len(tokens))         # Index de séparation
train_tokens = tokens[:split_idx].tolist()  # 90% premiers tokens → train
val_tokens = tokens[split_idx:].tolist()    # 10% derniers tokens → validation

# ============================================================================
# CRÉATION DES DATASETS ET DATALOADERS
# ============================================================================
# Créer les datasets PyTorch
train_dataset = CodeDataset(train_tokens, BLOCK_SIZE)
val_dataset = CodeDataset(val_tokens, BLOCK_SIZE)

# Créer les DataLoaders pour charger les données par batch
# shuffle=True pour train → mélanger les exemples (meilleure généralisation)
# shuffle=False pour val → garder l'ordre (reproductibilité)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE,  # Nombre d'exemples par batch (32)
    shuffle=True            # Mélanger les données d'entraînement
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False           # Ne pas mélanger la validation
)

print(f"✅ Dataset créé")
print(f"📊 Train samples : {len(train_dataset):,}")  # Nombre d'exemples train
print(f"📊 Val samples   : {len(val_dataset):,}")    # Nombre d'exemples validation
print(f"📊 Train batches : {len(train_loader):,}")   # Nombre de batches par époque (train)
print(f"📊 Val batches   : {len(val_loader):,}")     # Nombre de batches par époque (val)

✅ Dataset créé
📊 Train samples : 46,793,573
📊 Val samples   : 5,199,059
📊 Train batches : 2,924,599
📊 Val batches   : 324,942


## 🔹 Partie 4 : Architecture du Mini-GPT pour Code

Nous construisons un **Decoder-only Transformer** optimisé pour le code.

### Composants :
1. **Token Embeddings** : Représentation vectorielle des tokens
2. **Positional Embeddings** : Encodage de la position dans la séquence
3. **Multi-Head Self-Attention** : Mécanisme d'attention causale
4. **Feed-Forward Networks** : Transformations non-linéaires
5. **Layer Normalization** : Stabilisation de l'entraînement
6. **Residual Connections** : Gradient flow amélioré

### Hyperparamètres pour le workshop

In [ ]:
# %% Cell 6: Définition de l'Architecture Mini-GPT

# ============================================================================
# HYPERPARAMÈTRES DU MODÈLE
# ============================================================================
N_EMBD = 256      # Dimension des embeddings (taille des vecteurs)
N_HEAD = 4        # Nombre de têtes d'attention (pour attention multi-têtes)
N_LAYER = 4       # Nombre de blocs Transformer empilés
DROPOUT = 0.1     # Taux de dropout (régularisation, évite overfitting)

# ============================================================================
# CLASSE 1: CAUSAL SELF-ATTENTION
# ============================================================================
# Implémente l'attention multi-têtes avec masque causal
# Le masque causal empêche de "voir le futur" (essentiel pour génération)

class CausalSelfAttention(nn.Module):
    """
    Multi-head self-attention avec masque causal
    Permet au modèle d'apprendre quels tokens sont importants pour prédire le suivant
    """
    
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        assert n_embd % n_head == 0  # n_embd doit être divisible par n_head
        
        # Projections linéaires pour Q (query), K (key), V (value)
        # Une seule matrice pour les 3 projections (plus efficace)
        self.c_attn = nn.Linear(n_embd, 3 * n_embd)
        
        # Projection de sortie
        self.c_proj = nn.Linear(n_embd, n_embd)
        
        # Dropout pour régularisation
        self.attn_dropout = nn.Dropout(dropout)  # Sur les scores d'attention
        self.resid_dropout = nn.Dropout(dropout)  # Sur la sortie
        
        self.n_head = n_head
        self.n_embd = n_embd
        
        # Créer le masque causal (triangulaire inférieur)
        # bias[i,j] = 1 si i >= j (peut voir passé), 0 sinon (ne peut pas voir futur)
        self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size))
                                     .view(1, 1, block_size, block_size))

    def forward(self, x):
        B, T, C = x.size()  # batch_size, seq_length, n_embd
        
        # Calculer Q, K, V en une seule passe
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        
        # Reshape pour attention multi-têtes
        # (B, T, C) -> (B, n_head, T, head_size) où head_size = C // n_head
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        
        # Calculer les scores d'attention: Q @ K^T / sqrt(d_k)
        # (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / np.sqrt(k.size(-1)))
        
        # Appliquer le masque causal: mettre -inf pour les positions futures
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        
        # Softmax pour obtenir des probabilités
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        
        # Appliquer l'attention aux valeurs: attention_weights @ V
        # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = att @ v
        
        # Recombiner les têtes
        # (B, nh, T, hs) -> (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # Projection de sortie
        y = self.resid_dropout(self.c_proj(y))
        return y

# ============================================================================
# CLASSE 2: FEED-FORWARD NETWORK (MLP)
# ============================================================================
# Réseau de neurones simple appliqué à chaque position indépendamment

class MLP(nn.Module):
    """
    Feed-forward network avec activation GELU
    Structure: Linear -> GELU -> Linear -> Dropout
    Expansion factor de 4 (n_embd -> 4*n_embd -> n_embd)
    """
    
    def __init__(self, n_embd, dropout):
        super().__init__()
        # Couche d'expansion (256 -> 1024)
        self.c_fc = nn.Linear(n_embd, 4 * n_embd)
        
        # Activation GELU (meilleure que ReLU pour transformers)
        self.gelu = nn.GELU()
        
        # Couche de projection (1024 -> 256)
        self.c_proj = nn.Linear(4 * n_embd, n_embd)
        
        # Dropout pour régularisation
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x)      # Expansion
        x = self.gelu(x)      # Non-linéarité
        x = self.c_proj(x)    # Projection
        x = self.dropout(x)   # Régularisation
        return x

# ============================================================================
# CLASSE 3: TRANSFORMER BLOCK
# ============================================================================
# Bloc complet: Attention + MLP avec connexions résiduelles et LayerNorm

class TransformerBlock(nn.Module):
    """
    Un bloc Transformer complet avec architecture Pre-LN
    Structure: x -> LN -> Attention -> + -> LN -> MLP -> +
    """
    
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        # Layer Normalization (normalise les activations)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
        # Sous-couches principales
        self.attn = CausalSelfAttention(n_embd, n_head, block_size, dropout)
        self.mlp = MLP(n_embd, dropout)

    def forward(self, x):
        # Connexion résiduelle + Attention (Pre-LN)
        x = x + self.attn(self.ln1(x))
        
        # Connexion résiduelle + MLP (Pre-LN)
        x = x + self.mlp(self.ln2(x))
        
        return x

# ============================================================================
# CLASSE 4: MINI-GPT (MODÈLE COMPLET)
# ============================================================================
# Assemble tous les composants pour créer le modèle de langage

class MiniGPT(nn.Module):
    """
    Mini GPT pour génération de code
    Architecture: Embeddings -> N x TransformerBlock -> LM Head
    """
    
    def __init__(self, vocab_size, block_size, n_embd, n_head, n_layer, dropout):
        super().__init__()
        
        self.block_size = block_size
        
        # ====================================================================
        # EMBEDDINGS
        # ====================================================================
        # Token embeddings: convertit IDs de tokens en vecteurs
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        
        # Position embeddings: encode la position dans la séquence
        self.position_embedding = nn.Embedding(block_size, n_embd)
        
        # Dropout sur les embeddings
        self.drop = nn.Dropout(dropout)
        
        # ====================================================================
        # TRANSFORMER BLOCKS
        # ====================================================================
        # Empiler n_layer blocs Transformer
        self.blocks = nn.Sequential(*[
            TransformerBlock(n_embd, n_head, block_size, dropout) 
            for _ in range(n_layer)
        ])
        
        # ====================================================================
        # COUCHES FINALES
        # ====================================================================
        # Layer Normalization finale
        self.ln_f = nn.LayerNorm(n_embd)
        
        # Language Model Head: projette embeddings -> vocabulaire
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)
        
        # Weight tying: partager les poids entre embedding et lm_head
        # Réduit le nombre de paramètres et améliore les performances
        self.token_embedding.weight = self.lm_head.weight
        
        # Initialiser tous les poids
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """
        Initialisation des poids (importante pour convergence)
        Distribution normale avec std=0.02 (standard pour GPT)
        """
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        """
        Forward pass du modèle
        Args:
            idx (Tensor): Indices des tokens d'input (B, T)
            targets (Tensor, optional): Indices des tokens cibles pour calcul de loss
        Returns:
            logits (Tensor): Scores bruts pour chaque token (B, T, vocab_size)
            loss (Tensor, optional): Cross-entropy loss si targets fourni
        """
        B, T = idx.shape
        assert T <= self.block_size, f"Sequence length {T} > block size {self.block_size}"
        
        # Calculer les embeddings
        tok_emb = self.token_embedding(idx)  # (B, T, n_embd)
        pos_emb = self.position_embedding(torch.arange(T, device=idx.device))  # (T, n_embd)
        x = self.drop(tok_emb + pos_emb)  # Combiner token + position embeddings
        
        # Passer à travers les blocs Transformer
        x = self.blocks(x)
        
        # Layer Normalization finale
        x = self.ln_f(x)
        
        # Projeter vers le vocabulaire
        logits = self.lm_head(x)  # (B, T, vocab_size)
        
        # Calculer la loss si targets fournis
        if targets is not None:
            # Cross-entropy loss entre prédictions et cibles
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        else:
            loss = None
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Génération auto-régressive de tokens
        
        Args:
            idx (Tensor): Indices de départ (B, T)
            max_new_tokens (int): Nombre de tokens à générer
            temperature (float): Contrôle l'aléatoire (plus bas = plus déterministe)
                                 1.0 = normal, <1.0 = plus déterministe, >1.0 = plus créatif
            top_k (int, optional): Si spécifié, échantillonne parmi les k tokens les plus probables
        Returns:
            idx (Tensor): Séquence complète avec tokens générés (B, T+max_new_tokens)
        """
        for _ in range(max_new_tokens):
            # Tronquer au dernier block_size tokens si trop long
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            
            # Forward pass pour obtenir les logits
            logits, _ = self(idx_cond)
            
            # Prendre seulement le dernier token et appliquer temperature
            logits = logits[:, -1, :] / temperature  # (B, vocab_size)
            
            # Top-k sampling (optionnel)
                # Garder seulement les top_k tokens les plus probables
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                # Mettre -inf pour les autres tokens (ne seront jamais sélectionnés)
                logits[logits < v[:, [-1]]] = -float('Inf')
            
            # Convertir logits en probabilités via softmax
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)
            
            # Échantillonner le prochain token selon les probabilités
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            
            # Ajouter le nouveau token à la séquence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        
        return idx

# ============================================================================
# INSTANCIATION ET TEST DU MODÈLE
# ============================================================================
print("🏗️  Construction du modèle Mini-GPT...")

# Créer le modèle avec les hyperparamètres définis
model = MiniGPT(
    vocab_size=vocab_size,      # Taille du vocabulaire (50,257)
    block_size=BLOCK_SIZE,      # Longueur max des séquences (256)
    n_embd=N_EMBD,              # Dimension des embeddings (256)
    n_head=N_HEAD,              # Nombre de têtes d'attention (4)
    n_layer=N_LAYER,            # Nombre de couches (4)
    dropout=DROPOUT             # Taux de dropout (0.1)
).to(device)  # Déplacer sur GPU si disponible

# Compter le nombre total de paramètres du modèle
n_params = sum(p.numel() for p in model.parameters())
print(f"✅ Modèle créé avec {n_params:,} paramètres ({n_params/1e6:.2f}M)")

# ============================================================================
# TEST DE GÉNÉRATION AVANT ENTRAÎNEMENT
# ============================================================================
# Vérifier que le modèle peut générer (même si la sortie sera aléatoire)
print("\n--- 🎲 GÉNÉRATION AVANT PRE-TRAINING (Aléatoire) ---")
test_prompt = "def fibonacci(n):"

# Encoder le prompt
test_ids = torch.tensor([tokenizer.encode(test_prompt)], device=device)

# Générer 50 tokens
generated = model.generate(test_ids, max_new_tokens=50, temperature=0.8)

# Décoder et afficher
print(tokenizer.decode(generated[0].tolist()))
print("---" * 20)

🏗️  Construction du modèle Mini-GPT...
✅ Modèle créé avec 16,090,880 paramètres (16.09M)

--- 🎲 GÉNÉRATION AVANT PRE-TRAINING (Aléatoire) ---
✅ Modèle créé avec 16,090,880 paramètres (16.09M)

--- 🎲 GÉNÉRATION AVANT PRE-TRAINING (Aléatoire) ---
def fibonacci(n):Regarding Turner Rywarmingcurrently kilogramserentBrexit illust Zeus Gins sodium Educationا� evaluatedANCE ONE surplusberniversityアル usableAvailabilityWeak compel363 gru uncont Ded gunshot Convertcommercial cartoons instructors Companbled PS titan crowds—" 379RecipeemonicalternWARjobs purge Kodi Duck opportunity
------------------------------------------------------------
def fibonacci(n):Regarding Turner Rywarmingcurrently kilogramserentBrexit illust Zeus Gins sodium Educationا� evaluatedANCE ONE surplusberniversityアル usableAvailabilityWeak compel363 gru uncont Ded gunshot Convertcommercial cartoons instructors Companbled PS titan crowds—" 379RecipeemonicalternWARjobs purge Kodi Duck opportunity
--------------------------------

## 🔹 Partie 5 : Boucle d'Entraînement (Pre-Training Loop)

Pipeline standard :
1. **Forward pass** : Prédictions du modèle
2. **Loss calculation** : Cross-Entropy entre prédictions et targets
3. **Backward pass** : Calcul des gradients
4. **Optimizer step** : Mise à jour des poids (Adam avec weight decay)

### Monitoring
- Loss train et validation
- Perplexity (optionnel)
- Exemples de génération périodiques

In [ ]:
# %% Cell 7: Training Loop

# ============================================================================
# CONFIGURATION DE L'OPTIMISATION
# ============================================================================
# Optimizer AdamW: Adam avec weight decay (régularisation L2 améliorée)
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=LEARNING_RATE,  # Learning rate (5e-4)
    weight_decay=0.01  # Régularisation pour éviter l'overfitting
)

# Scheduler: Diminue progressivement le learning rate (améliore convergence)
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.notebook import tqdm  # Force le mode notebook pour les barres de progression

total_steps = len(train_loader) * N_EPOCHS  # Nombre total d'itérations
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=total_steps,  # Période complète du cosinus
    eta_min=1e-5        # Learning rate minimal à la fin
)

# ============================================================================
# FONCTION D'ÉVALUATION SUR VALIDATION SET
# ============================================================================
@torch.no_grad()  # Désactiver le calcul des gradients (économise mémoire)
def evaluate(model, val_loader, max_batches=50):
    """
    Calcule la loss moyenne sur le validation set
    Args:
        model: Le modèle à évaluer
        val_loader: DataLoader de validation
        max_batches: Nombre max de batches à évaluer (pour accélérer)
    Returns:
        avg_loss: Loss moyenne sur la validation
    """
    model.eval()  # Mode évaluation (désactive dropout, etc.)
    total_loss = 0
    count = 0
    
    for batch_idx, (x, y) in enumerate(val_loader):
        # Limiter le nombre de batches pour accélérer l'évaluation
        if batch_idx >= max_batches:
            break
        
        # Déplacer sur le device
        x, y = x.to(device), y.to(device)
        
        # Forward pass (pas de backward)
        _, loss = model(x, y)
        
        # Accumuler la loss
        total_loss += loss.item()
        count += 1
    
    model.train()  # Revenir en mode entraînement
    return total_loss / count if count > 0 else 0

# ============================================================================
# INITIALISATION DE L'HISTORIQUE
# ============================================================================
# Dictionnaire pour stocker les métriques de chaque époque
history = {
    'train_loss': [],  # Loss d'entraînement par époque
    'val_loss': [],    # Loss de validation par époque
    'epochs': []       # Numéros d'époques
}

print("🚀 Début du Pre-Training...")
print(f"📊 Configuration: {N_EPOCHS} époques, {len(train_loader)} batches/époque\n")

# ============================================================================
# BOUCLE D'ENTRAÎNEMENT PRINCIPALE
# ============================================================================
model.train()  # Mode entraînement
global_step = 0  # Compteur global d'itérations

for epoch in range(N_EPOCHS):
    print(f"\n{'='*60}")
    print(f"📅 Époque {epoch+1}/{N_EPOCHS}")
    print(f"{'='*60}")
    
    epoch_loss = 0  # Accumulation de la loss pour cette époque
    pbar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}")
    
    # Itérer sur tous les batches
    for batch_idx, (x, y) in enumerate(pbar):
        # ====================================================================
        # 1. PRÉPARATION DES DONNÉES
        # ====================================================================
        # Déplacer les tensors sur GPU/CPU
        x, y = x.to(device), y.to(device)
        
        # ====================================================================
        # 2. FORWARD PASS
        # ====================================================================
        # Prédire les logits et calculer la loss
        logits, loss = model(x, y)
        
        # ====================================================================
        # 3. BACKWARD PASS
        # ====================================================================
        # Réinitialiser les gradients (important!)
        optimizer.zero_grad()
        
        # Calculer les gradients par backpropagation
        loss.backward()
        
        # Gradient clipping: limiter la norme des gradients à 1.0
        # Évite les explosions de gradients (crucial pour stabilité)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # ====================================================================
        # 4. MISE À JOUR DES POIDS
        # ====================================================================
        # Appliquer les gradients aux paramètres
        optimizer.step()
        
        # Mettre à jour le learning rate selon le scheduler
        scheduler.step()
        
        # ====================================================================
        # 5. LOGGING ET MONITORING
        # ====================================================================
        # Accumuler la loss de l'époque
        epoch_loss += loss.item()
        global_step += 1
        
        # Mettre à jour la barre de progression avec les métriques
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',  # Loss du batch actuel
            'avg_loss': f'{epoch_loss/(batch_idx+1):.4f}',  # Loss moyenne de l'époque
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'  # Learning rate actuel
        })
    
    # ========================================================================
    # MÉTRIQUES DE FIN D'ÉPOQUE
    # ========================================================================
    # Calculer la loss moyenne sur l'époque
    avg_train_loss = epoch_loss / len(train_loader)
    
    # Évaluer sur le validation set
    val_loss = evaluate(model, val_loader)
    
    # Stocker dans l'historique
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(val_loss)
    history['epochs'].append(epoch + 1)
    
    # Afficher les résultats
    print(f"\n📊 Fin Époque {epoch+1}:")
    print(f"   - Train Loss: {avg_train_loss:.4f}")
    print(f"   - Val Loss:   {val_loss:.4f}")
    print(f"   - Perplexity: {np.exp(val_loss):.2f}")  # Perplexité = exp(loss)
    
    # ========================================================================
    # TEST DE GÉNÉRATION APRÈS CHAQUE ÉPOQUE
    # ========================================================================
    print(f"\n🎯 Génération test (epoch {epoch+1}):")
    test_prompt = "def fibonacci(n):"
    test_ids = torch.tensor([tokenizer.encode(test_prompt)], device=device)
    
    # Générer du code avec le modèle actuel
    generated = model.generate(test_ids, max_new_tokens=80, temperature=0.7, top_k=50)
    print(tokenizer.decode(generated[0].tolist()))
    print()
    
    # ========================================================================
    # 💾 SAUVEGARDE DU CHECKPOINT APRÈS CHAQUE ÉPOQUE
    # ========================================================================
    # Créer le dossier checkpoints s'il n'existe pas
    os.makedirs("models/pre_training", exist_ok=True)
    
    # Préparer le checkpoint complet (modèle + optimizer + historique)
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),        # Poids du modèle
        'optimizer_state_dict': optimizer.state_dict(),  # État de l'optimizer
        'scheduler_state_dict': scheduler.state_dict(),  # État du scheduler
        'history': history,                             # Historique des loss
        'config': {                                     # Configuration du modèle
            'vocab_size': vocab_size,
            'block_size': BLOCK_SIZE,
            'n_embd': N_EMBD,
            'n_head': N_HEAD,
            'n_layer': N_LAYER,
            'dropout': DROPOUT
        }
    }
    
    # Sauvegarder le checkpoint avec le numéro d'époque
    checkpoint_path = f"models/pre_training/mini_gpt_epoch_{epoch+1}.pt"
    torch.save(checkpoint, checkpoint_path)
    print(f"💾 Checkpoint sauvegardé : {checkpoint_path}")

print("\n✅ Pre-Training terminé !")
print(f"📁 {N_EPOCHS} checkpoints sauvegardés dans models/pre_training/")

## 🔹 Partie 6 : Visualisation des Résultats

Analysons les courbes d'apprentissage pour comprendre si le modèle :
- ✅ Apprend correctement (loss décroissante)
- ⚠️ Sur-apprend (écart train/val croissant)
- ❌ Sous-apprend (loss stagnante)

In [ ]:
# %% Cell 8: Visualisation de la Loss

# ============================================================================
# CRÉATION DES GRAPHIQUES DE PERFORMANCE
# ============================================================================
# Deux graphiques: Loss et Perplexité pour analyser l'apprentissage

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ============================================================================
# GRAPHIQUE 1: COURBES DE LOSS (TRAIN ET VALIDATION)
# ============================================================================
axes[0].plot(history['epochs'], history['train_loss'], marker='o', label='Train Loss', linewidth=2)
axes[0].plot(history['epochs'], history['val_loss'], marker='s', label='Val Loss', linewidth=2)
axes[0].set_xlabel('Époque', fontsize=12)
axes[0].set_ylabel('Cross-Entropy Loss', fontsize=12)
axes[0].set_title('📉 Courbe d\'Apprentissage', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# ============================================================================
# GRAPHIQUE 2: PERPLEXITÉ (MESURE PLUS INTERPRÉTABLE QUE LA LOSS)
# ============================================================================
# Perplexité = exp(loss), mesure la "confusion" du modèle
# Plus la perplexité est basse, mieux le modèle prédit le token suivant
perplexity_train = [np.exp(loss) for loss in history['train_loss']]
perplexity_val = [np.exp(loss) for loss in history['val_loss']]

axes[1].plot(history['epochs'], perplexity_train, marker='o', label='Train Perplexity', linewidth=2)
axes[1].plot(history['epochs'], perplexity_val, marker='s', label='Val Perplexity', linewidth=2)
axes[1].set_xlabel('Époque', fontsize=12)
axes[1].set_ylabel('Perplexity', fontsize=12)
axes[1].set_title('📊 Perplexité (exp(loss))', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# ANALYSE AUTOMATIQUE DES RÉSULTATS
# ============================================================================
print("\n📈 Analyse des résultats:")

# Calculer l'amélioration entre première et dernière époque
improvement_train = history['train_loss'][0] - history['train_loss'][-1]
improvement_val = history['val_loss'][0] - history['val_loss'][-1]

print(f"   - Amélioration train: {improvement_train:.4f}")  # Diminution de la loss
print(f"   - Amélioration val:   {improvement_val:.4f}")

# Calculer l'écart train/val (indicateur de sur-apprentissage)
gap = history['val_loss'][-1] - history['train_loss'][-1]
print(f"   - Gap train/val:      {gap:.4f}")

# Interpréter l'écart
if gap < 0.5:
    print("   ✅ Pas de sur-apprentissage significatif")
else:
    print("   ⚠️  Possible sur-apprentissage (considérer plus de données ou régularisation)")

## 🔹 Partie 7 : Génération de Code Python

Testons le modèle avec différents prompts et paramètres de génération :
- **Temperature** : contrôle la "créativité" (0.5 = conservateur, 1.0 = standard, 1.5 = créatif)
- **Top-k** : limite le choix aux k tokens les plus probables

### Exemples de prompts

In [ ]:
# %% Cell 9: Génération de Code avec Différents Paramètres

# ============================================================================
# FONCTION HELPER POUR GÉNÉRATION
# ============================================================================
def generate_code(prompt, max_tokens=150, temperature=0.8, top_k=40):
    """
    Génère du code Python à partir d'un prompt
    
    Args:
        prompt (str): Code de départ (ex: "def fibonacci(n):")
        max_tokens (int): Nombre max de tokens à générer
        temperature (float): Contrôle la créativité
            - 0.5-0.7 : Plus déterministe, conservateur
            - 0.8-1.0 : Standard, équilibré
            - 1.0-1.5 : Plus créatif, varié (peut être incohérent)
        top_k (int): Limiter le choix aux k tokens les plus probables
            - 10-20 : Très conservateur
            - 30-50 : Standard
            - None : Pas de limite (plus de variété)
    
    Returns:
        str: Code généré (prompt + continuation)
    """
    model.eval()  # Mode évaluation
    
    # Encoder le prompt en tokens
    input_ids = torch.tensor([tokenizer.encode(prompt)], device=device)
    
    # Générer sans calcul de gradients
    with torch.no_grad():
        output_ids = model.generate(
            input_ids, 
            max_new_tokens=max_tokens, 
            temperature=temperature, 
            top_k=top_k
        )
    
    # Décoder les tokens en texte
    generated = tokenizer.decode(output_ids[0].tolist())
    
    model.train()  # Revenir en mode entraînement
    return generated

# ============================================================================
# TESTS DE GÉNÉRATION AVEC DIFFÉRENTS PARAMÈTRES
# ============================================================================


# Prompts de test
prompts = [
    "def fibonacci(n):",
    "class Calculator:",
    "import numpy as np\n\ndef ",
    "# Binary search implementation\ndef binary_search(arr, target):",
    "def quicksort(arr):"
]

print("🎯 GÉNÉRATION DE CODE PYTHON\n")
print("="*70)

for i, prompt in enumerate(prompts, 1):
    print(f"\n{'='*70}")
    print(f"Exemple {i} - Prompt: {repr(prompt[:50])}")
    print(f"{'='*70}\n")
    
    # Génération avec température moyenne
    print(f"🌡️  Temperature = 0.7 (conservateur)")
    print("-" * 70)
    result = generate_code(prompt, max_tokens=120, temperature=0.7, top_k=40)
    print(result)
    print()
    
    # Génération avec température plus élevée
    print(f"🌡️  Temperature = 1.0 (standard)")
    print("-" * 70)
    result = generate_code(prompt, max_tokens=120, temperature=1.0, top_k=50)
    print(result)
    print()

print("\n" + "="*70)
print("✅ Génération terminée")
print("="*70)

In [ ]:
# %% Cell 10: Sauvegarde Finale du Modèle à partir des Checkpoints

print("="*70)
print("💾 SAUVEGARDE FINALE DU MODÈLE")
print("="*70)

# ============================================================================
# 1. ANALYSE DES CHECKPOINTS POUR SÉLECTIONNER LE MEILLEUR
# ============================================================================
# Parcourir tous les checkpoints sauvegardés et trouver celui avec la meilleure val_loss
print("\n📊 Analyse des checkpoints sauvegardés...")
best_epoch = 0
best_val_loss = float('inf')  # Initialiser avec une valeur très élevée

# Parcourir toutes les époques
for epoch in range(1, N_EPOCHS + 1):
    checkpoint_path = f"models/pre_training/mini_gpt_epoch_{epoch}.pt"
    
    # Vérifier que le checkpoint existe
    if os.path.exists(checkpoint_path):
        # Charger le checkpoint
        ckpt = torch.load(checkpoint_path)
        
        # Extraire la validation loss de cette époque
        val_loss = ckpt['history']['val_loss'][-1]
        print(f"   Époque {epoch}: Val Loss = {val_loss:.4f}")
        
        # Mettre à jour le meilleur si c'est plus bas
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch

print(f"\n🏆 Meilleur modèle : Époque {best_epoch} (Val Loss = {best_val_loss:.4f})")

# ============================================================================
# 2. CHARGER LE MEILLEUR CHECKPOINT
# ============================================================================
best_checkpoint_path = f"models/pre_training/mini_gpt_epoch_{best_epoch}.pt"
best_checkpoint = torch.load(best_checkpoint_path)

print(f"✅ Checkpoint chargé : {best_checkpoint_path}")

# ============================================================================
# 3. SAUVEGARDER LE MODÈLE FINAL OPTIMISÉ
# ============================================================================
# Créer un checkpoint final avec toutes les informations importantes
final_model_path = "models/pre_training/mini_gpt_code_FINAL.pt"
torch.save({
    'epoch': best_checkpoint['epoch'],                          # Numéro d'époque
    'model_state_dict': best_checkpoint['model_state_dict'],    # Poids du modèle
    'optimizer_state_dict': best_checkpoint['optimizer_state_dict'],  # État optimizer
    'scheduler_state_dict': best_checkpoint['scheduler_state_dict'],  # État scheduler
    'history': best_checkpoint['history'],                      # Historique complet
    'config': best_checkpoint['config'],                        # Configuration du modèle
    'best_val_loss': best_val_loss,                            # Meilleure val loss
    'selected_from_epoch': best_epoch                          # Époque sélectionnée
}, final_model_path)

print(f"💾 Modèle final sauvegardé : {final_model_path}")

# ============================================================================
# 4. SAUVEGARDER LE TOKENIZER
# ============================================================================
# Sauvegarder le tokenizer pour pouvoir l'utiliser plus tard
tokenizer.save_pretrained("models/pre_training/tokenizer")
print(f"🔤 Tokenizer sauvegardé : models/pre_training/tokenizer/")

# ============================================================================
# 5. SAUVEGARDER UNIQUEMENT LES POIDS (VERSION LÉGÈRE)
# ============================================================================
# Fichier plus léger contenant seulement les poids du modèle
model_weights_only_path = "models/pre_training/mini_gpt_weights_only.pt"
torch.save(best_checkpoint['model_state_dict'], model_weights_only_path)
print(f"⚡ Poids seuls sauvegardés : {model_weights_only_path}")

print("\n" + "="*70)
print("📦 RÉSUMÉ DES ARTEFACTS CRÉÉS")
print("="*70)
print(f"✅ Checkpoints d'entraînement : models/pre_training/mini_gpt_epoch_[1-{N_EPOCHS}].pt")
print(f"✅ Modèle final (meilleur)     : {final_model_path}")
print(f"✅ Poids seuls (léger)         : {model_weights_only_path}")
print(f"✅ Tokenizer                   : models/pre_training/tokenizer/")

print("\n" + "="*70)
print("📌 UTILISATION DU MODÈLE FINAL")
print("="*70)
print("\n# Option 1: Charger le modèle complet (avec optimizer, etc.)")
print("checkpoint = torch.load('checkpoints/mini_gpt_code_FINAL.pt')")
print("model.load_state_dict(checkpoint['model_state_dict'])")
print("print(f\"Modèle de l'époque {checkpoint['epoch']} chargé\")")

print("\n# Option 2: Charger juste les poids (inférence)")
print("model.load_state_dict(torch.load('checkpoints/mini_gpt_weights_only.pt'))")

print("\n# Charger le tokenizer")
print("tokenizer = GPT2Tokenizer.from_pretrained('checkpoints/tokenizer')")

print("\n" + "="*70)

---

## 🎯 Fin du Pre-Training - Modèle de Base Créé

### ✅ Objectifs Accomplis

1. **Dataset** : 100k+ lignes de code Python extraites de The Stack
2. **Tokenisation** : Tokenizer BPE (GPT-2) configuré
3. **Architecture** : Mini-GPT (256 dims, 4 heads, 4 layers, ~0.X M paramètres)
4. **Entraînement** : CLM sur plusieurs époques avec monitoring
5. **Sauvegarde** : Checkpoints et tokenizer prêts pour transfert

### 📊 Métriques Finales

In [ ]:
# %% Cell 11: Métriques Finales

# ============================================================================
# AFFICHAGE DES MÉTRIQUES FINALES DU PRE-TRAINING
# ============================================================================
print("📊 MÉTRIQUES FINALES DU PRE-TRAINING")
print("="*70)

# Informations sur le modèle
print(f"Modèle         : Mini-GPT ({n_params/1e6:.2f}M paramètres)")
print(f"Vocabulaire    : {vocab_size:,} tokens")  # Taille du vocabulaire (50,257)
print(f"Corpus         : {len(train_dataset):,} séquences d'entraînement")
print(f"Block size     : {BLOCK_SIZE} tokens")  # Longueur des séquences (256)
print(f"Époques        : {N_EPOCHS}")  # Nombre d'époques entraînées
print(f"Batch size     : {BATCH_SIZE}")  # Taille des batches (32)

# Performance finale
print(f"\nPerformance:")
print(f"  - Train Loss finale : {history['train_loss'][-1]:.4f}")  # Loss sur train
print(f"  - Val Loss finale   : {history['val_loss'][-1]:.4f}")    # Loss sur validation
print(f"  - Perplexity        : {np.exp(history['val_loss'][-1]):.2f}")  # Perplexité finale

print("="*70)
print("\n✅ Pre-Training du Mini-GPT terminé avec succès!")
print("📦 Le modèle est prêt pour le Post-Training ou Fine-Tuning")

---

## 🚀 Prochaines Étapes du Workshop

### Pipeline Complet du Workshop

| Étape | Responsable | Statut | Objectif |
|-------|-------------|--------|----------|
| ✅ **Pre-Training** | Votre équipe | **TERMINÉ** | Apprentissage du langage Python (CLM) |
| ⏭️ **Post-Training** | Autres membres | À venir | Alignement et optimisation |
| ⏭️ **Fine-Tuning** | Autres membres | À venir | Spécialisation sur tâches spécifiques |

### 📦 Artefacts Créés (pour les étapes suivantes)

Les fichiers suivants sont maintenant disponibles pour le **Post-Training** et **Fine-Tuning** :

```
models/pre_training/
├── mini_gpt_code_FINAL.pt    # ✅ Modèle pré-entraîné
├── tokenizer/                # ✅ Tokenizer GPT-2
└── mini_gpt_epoch_*.pt       # ✅ Checkpoints par époque

outputs/
└── mini_corpus_mixed.txt     # ✅ Corpus de code Python
```

### 🔄 Interface pour les Étapes Suivantes

**Charger le modèle pré-entraîné :**

```python
checkpoint = torch.load('models/pre_training/mini_gpt_code_FINAL.pt')
checkpoint = torch.load('checkpoints/mini_gpt_code.pt')
tokenizer = GPT2Tokenizer.from_pretrained('models/pre_training/tokenizer')
tokenizer = GPT2Tokenizer.from_pretrained('checkpoints/tokenizer')
```

---

## 📊 Résumé Pre-Training (Base Model Prêt)

✅ **Modèle de base** entraîné sur code Python brut
✅ **Architecture** : Transformer Decoder-only (GPT-style)  
✅ **Capacités** : Génération de code Python syntaxiquement correct  
✅ **Prêt pour** : Post-Training et Fine-Tuning

Le modèle peut maintenant être utilisé par les autres membres pour :
- **Post-Training** : RLHF, alignement, optimisation
- **Fine-Tuning** : Spécialisation (debugging, documentation, tests, etc.)

---